In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import torch

In [ ]:
is_cuda = False
if torch.cuda.is_available():
    is_cuda = True

# Load data

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder

simple_transform = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
train = ImageFolder('C:/Users/Yeonkang/Desktop/Deep_Learning/Image_Recognition/Vanilla_CNN/Python/data/dogsandcats/trainandvalid/train', simple_transform)
valid = ImageFolder('C:/Users/Yeonkang/Desktop/Deep_Learning/Image_Recognition/Vanilla_CNN/Python/data/dogsandcats/trainandvalid/valid', simple_transform)

In [ ]:
train_data_gen = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True, num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid, batch_size=16, shuffle=True, num_workers=3)

# Adjust VGG16

## Change *out_features* of the last linear layer

In [ ]:
from torchvision import models
vgg = models.vgg16(pretrained=True)

In [ ]:
vgg

In [ ]:
for param in vgg.features.parameters():
    param.requires_grad = False
    
vgg.classifier[6].out_features = 2

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F

def fit(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data, target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        
        running_loss += F.cross_entropy(output, target, reduction='mean').data
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss.item()/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [ ]:
import torch.optim as optim

vgg = vgg.cuda()

optimizer = optim.SGD(vgg.classifier.parameters(), lr=0.0001, momentum=0.5)
train_losses,train_accuracy = [],[]
val_losses,val_accuracy = [],[]

for epoch in range(1,10):
    train_epoch_loss,train_epoch_accuracy = fit(epoch, vgg, train_data_gen, phase='training')
    val_epoch_loss,val_epoch_accuracy = fit(epoch, vgg, valid_data_gen, phase='validation')
    train_losses.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

In [ ]:
plt.plot(range(1,len(train_losses)+1),train_losses, 'bo', label='train')
plt.plot(range(1,len(val_losses)+1),val_losses, 'r', label='validation')
plt.title('Loss')
plt.legend()

In [ ]:
plt.plot(range(1,len(train_accuracy)+1),train_accuracy, 'bo', label='train')
plt.plot(range(1,len(val_accuracy)+1),val_accuracy, 'r', label='validation')
plt.title('Accuracy')
plt.legend()

## Change *p* of dropout

In [ ]:
import torch.nn as nn

for layer in vgg.classifier.children():
    if (type(layer)==nn.Dropout):
        layer.p = 0.2

In [ ]:
train_losses,train_accuracy = [],[]
val_losses,val_accuracy = [],[]

for epoch in range(1,3):
    train_epoch_loss,train_epoch_accuracy = fit(epoch, vgg, train_data_gen, phase='training')
    val_epoch_loss,val_epoch_accuracy = fit(epoch, vgg, valid_data_gen, phase='validation')
    train_losses.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

## Augument data

In [ ]:
def imshow(inp):
    inp = inp.numpy().transpose((1,2,0))
    mean,std = np.array([0.485,0.456,0.406]),np.array([0.229,0.224,0.225])
    inp = std * inp + mean
    inp = np.clip(inp,0,1)
    plt.imshow(inp)

In [ ]:
imshow(valid[50][0]) #original data

In [ ]:
sample_transform1 = transforms.Compose([transforms.Resize((224,224)), transforms.RandomHorizontalFlip(p=1), transforms.ToTensor(),
                                        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
valid = ImageFolder('C:/Users/Yeonkang/Desktop/Deep_Learning/Image_Recognition/Vanilla_CNN/Python/data/dogsandcats/trainandvalid/valid', sample_transform1)
imshow(valid[50][0]) #horizontally flipped image

In [ ]:
sample_transform2 = transforms.Compose([transforms.Resize((224,224)), transforms.RandomRotation(180), transforms.ToTensor(),
                                        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
valid = ImageFolder('C:/Users/Yeonkang/Desktop/Deep_Learning/Image_Recognition/Vanilla_CNN/Python/data/dogsandcats/trainandvalid/valid', sample_transform2)
imshow(valid[50][0]) #image rotated by 180 degree

- Code for explaining how do *RandomHorizontalFlip* and *RandomRotation* work

In [ ]:
train_transform = transforms.Compose([transforms.Resize((224,224)), transforms.RandomHorizontalFlip(),
                                     transforms.RandomRotation(0.2), transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
train = ImageFolder('C:/Users/Yeonkang/Desktop/Deep_Learning/Image_Recognition/Vanilla_CNN/Python/data/dogsandcats/trainandvalid/train', train_transform)
valid = ImageFolder('C:/Users/Yeonkang/Desktop/Deep_Learning/Image_Recognition/Vanilla_CNN/Python/data/dogsandcats/trainandvalid/valid', train_transform)

In [ ]:
train_data_gen = torch.utils.data.DataLoader(train, batch_size=16, shuffle=True, num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid, batch_size=16, shuffle=True, num_workers=3)

In [ ]:
train_losses,train_accuracy = [],[]
val_losses,val_accuracy = [],[]

for epoch in range(1,3):
    train_epoch_loss,train_epoch_accuracy = fit(epoch, vgg, train_data_gen, phase='training')
    val_epoch_loss,val_epoch_accuracy = fit(epoch, vgg, valid_data_gen, phase='validation')
    train_losses.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

## Calculate pre-convoluted features

In [ ]:
vgg = models.vgg16(pretrained=True)
vgg = vgg.cuda()

In [ ]:
features = vgg.features
for param in features.parameters(): 
    param.requires_grad = False

In [ ]:
train_data_gen = torch.utils.data.DataLoader(train, batch_size=16, num_workers=3, shuffle=False)
valid_data_gen = torch.utils.data.DataLoader(valid, batch_size=16, num_workers=3, shuffle=False)

In [ ]:
def preconvfeat(dataset, model):
    conv_features = []
    labels_list = []
    for data in dataset:
        inputs,labels = data
        if is_cuda:
            inputs, labels = inputs.cuda(),labels.cuda()       
        inputs, labels = Variable(inputs),Variable(labels)
        output = model(inputs)
        conv_features.extend(output.data.cpu().numpy())
        labels_list.extend(labels.data.cpu().numpy())
    conv_features = np.concatenate([[feat] for feat in conv_features])
    
    return (conv_features,labels_list)

In [ ]:
conv_feat_train,labels_train= preconvfeat(train_data_gen, features)
conv_feat_val,labels_val= preconvfeat(valid_data_gen, features)

In [ ]:
print(conv_feat_train.shape)
print(np.array(labels_train).shape)

In [ ]:
from torch.utils.data import Dataset

class My_dataset(Dataset):
    def __init__(self,feat,labels):
        self.conv_feat = feat
        self.labels = labels
    
    def __len__(self):
        return len(self.conv_feat)
    
    def __getitem__(self,idx):
        return self.conv_feat[idx],self.labels[idx]

In [ ]:
train_feat_dataset = My_dataset(conv_feat_train, labels_train)
val_feat_dataset = My_dataset(conv_feat_val, labels_val)

In [ ]:
train_feat_gen = torch.utils.data.DataLoader(train_feat_dataset, batch_size=16, shuffle=True)
val_feat_gen = torch.utils.data.DataLoader(val_feat_dataset, batch_size=16, shuffle=True)

In [ ]:
def fit_numpy(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , (data,target) in enumerate(data_loader):
        if is_cuda:
            data,target = data.cuda(),target.cuda()
        data , target = Variable(data,volatile),Variable(target)
        if phase == 'training':
            optimizer.zero_grad()
        data = data.view(data.size(0), -1)
        output = model(data)
        loss = F.cross_entropy(output,target)
        
        running_loss += F.cross_entropy(output, target, reduction='sum').data
        preds = output.data.max(dim=1, keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss/len(data_loader.dataset)
    accuracy = 100. * running_correct.item()/len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [ ]:
train_losses,train_accuracy = [],[]
val_losses,val_accuracy = [],[]
for epoch in range(1,20):
    epoch_loss,epoch_accuracy = fit_numpy(epoch, vgg.classifier, train_feat_gen, phase='training')
    val_epoch_loss ,val_epoch_accuracy = fit_numpy(epoch, vgg.classifier, val_feat_gen, phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

In [ ]:
plt.plot(range(1,len(train_losses)+1),train_losses, 'bo', label='train')
plt.plot(range(1,len(val_losses)+1),val_losses, 'r', label='validation')
plt.title('Loss')
plt.legend()

In [ ]:
plt.plot(range(1,len(train_accuracy)+1),train_accuracy, 'bo', label='train')
plt.plot(range(1,len(val_accuracy)+1),val_accuracy, 'r', label='validation')
plt.title('Accuracy')
plt.legend()

# Visualize outputs from intermediate layers

In [ ]:
train_data_gen = torch.utils.data.DataLoader(train,batch_size=16, num_workers=3, shuffle=False)
img,label = next(iter(train_data_gen))

In [ ]:
imshow(img[5])

In [ ]:
img = img[5][None]

In [ ]:
vgg = models.vgg16(pretrained=True).cuda()

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F

In [ ]:
class LayerActivations():
    features=None
    
    def __init__(self,model,layer_num):
        self.hook = model[layer_num].register_forward_hook(self.hook_fn)
    
    def hook_fn(self,module,input,output):
        self.features = output.cpu().data.numpy()
    
    def remove(self):
        self.hook.remove()

In [ ]:
conv_out = LayerActivations(vgg.features,0)
o = vgg(Variable(img.cuda()))
conv_out.remove()
act = conv_out.features

In [ ]:
fig = plt.figure(figsize=(20,50))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.8, hspace=0, wspace=0.2)
for i in range(30):
    ax = fig.add_subplot(12, 5, i+1, xticks=[], yticks=[])
    ax.imshow(act[0][i])

- Output of the first convolutional layer

In [ ]:
conv_out = LayerActivations(vgg.features,26)
o = vgg(Variable(img.cuda()))
conv_out.remove()
act = conv_out.features

In [ ]:
fig = plt.figure(figsize=(20,50))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.8, hspace=0, wspace=0.2)
for i in range(30):
    ax = fig.add_subplot(12, 5, i+1, xticks=[], yticks=[])
    ax.imshow(act[0][i])

In [ ]:
vgg = models.vgg16(pretrained=True).cuda()

In [ ]:
vgg.state_dict().keys()

In [ ]:
cnn_weights = vgg.state_dict()['features.0.weight'].cpu()

In [ ]:
fig = plt.figure(figsize=(30,30))
fig.subplots_adjust(left=0, right=1, bottom=0, top=0.8, hspace=0, wspace=0.2)
for i in range(30):
    ax = fig.add_subplot(12, 6, i+1, xticks=[], yticks=[])
    imshow(cnn_weights[i])